In [1]:
import numpy as np, pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,roc_curve
import plotly.express as px
from datetime import datetime
import time,random
#tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [0]:
train = pd.read_csv("https://raw.githubusercontent.com/Layden23/bet369/master/Datasets/train2.csv", header = 0)

In [0]:
random.seed(69)
del train["HostHandicapHT"]
del train["HostHandicapFT"]
del train["BettingCuoteHT"]
del train["BettingCuoteFT"]
del train["HostGoalsFT"]
del train["GuestGoalsFT"]

In [0]:
train = train[train['Status']== "HT"]
train = train.drop_duplicates(subset = "MatchID")
train = train.iloc[:-1]

In [5]:
train.sort_values(by = "MatchID")

,MatchID,LeagueID,LeagueName,LeagueFullname,LeagueAbbreviation,CountryID,CountryName,CountryAbbreviation,HomeTeamID,HomeTeamName,GuestTeamID,GuestTeamName,ExpectedGoalsFT,ExpectedGoalsHT,HostCornersFT,GuestCornersFT,HostYCFT,GuestYCFT,HostRCFT,GuestRCFT,HostAttacksFT,GuestAttacksFT,HostDAttacksFT,GuestDAttacksFT,HostonTargetFT,GuestonTargetFT,HostoffTargetFT,GuestoffTargetFT,HostPossessionFT,GuestPossessionFT,Status,Goal
29018,678625,35,Barclays PL,Barclays Premier League,B,14,England,Y,151,Man City,6387,Aston Villa,4.5,1.75,6,4,0,1,0,0,59,43,61,16,2,1,9,3,64,36,HT,0
54135,678626,35,Barclays PL,Barclays Premier League,B,14,England,Y,5778,Brighton,459,Everton,2.5,1.0,0,2,1,0,0,0,60,48,16,21,1,3,2,2,50,50,HT,1
53937,678627,35,Barclays PL,Barclays Premier League,B,14,England,Y,5933,Watford,1129,Bournemouth,2.75,1.25,1,6,1,0,0,0,66,40,19,23,1,3,3,5,42,58,HT,0
53740,678628,35,Barclays PL,Barclays Premier League,B,14,England,Y,6111,West Ham,6487,Sheff Utd,2.5,1.0,7,2,1,0,0,0,75,39,27,10,1,2,2,4,57,43,HT,1
122594,678630,35,Barclays PL,Barclays Premier League,B,14,England,Y,149,Newcastle,6039,Wolverhampton,2.0,0.75,2,3,0,0,0,0,49,60,29,17,1,2,8,1,50,50,HT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146081,688185,915,IND SPL,India Shillong Premier League,I,17,India,Y,22706,Rangdajied United FC,37414,Laban SC,4.75,2.0,1,0,2,0,0,0,69,43,49,28,3,0,7,2,49,51,HT,1
146613,688276,263,Friendlies,Friendlies,F,4,International,G,29113,Chulalongkorn University,7715,Thailand U19,3.0,1.25,3,2,0,0,0,0,34,48,21,23,0,1,4,1,0,0,HT,0
145628,688429,2555,Indonesia L3,Indonesia Liga 3,I,65,Indonesia,Y,1270,Persiba Bantul,3691,Persika Karawang,2.5,1.0,6,0,0,2,0,0,51,54,34,17,0,0,3,5,0,0,HT,0
145629,688461,2555,Indonesia L3,Indonesia Liga 3,I,65,Indonesia,Y,38475,Bina Putra,38522,Cirebon FC,2.75,1.0,3,2,1,2,0,0,51,38,12,5,0,0,0,1,0,0,HT,0


In [0]:
train = train.loc[(train['HostPossessionFT'] != "0") & (train['GuestPossessionFT'] != "0")]

In [7]:
train["Goal"] = train["Goal"].astype(str)
fig = px.histogram(train, x="Goal")
fig.show()

# Feature Engineering

In [8]:
X = train.iloc[:,12:30]
pca = PCA(0.99)
pca.fit(X)
pcX = pca.transform(X)
y = train["Goal"]
for i in X.columns:
    X[i] = pd.to_numeric(X[i])
y = pd.to_numeric(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.7)
X.dtypes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



ExpectedGoalsFT      float64
ExpectedGoalsHT      float64
HostCornersFT          int64
GuestCornersFT         int64
HostYCFT               int64
GuestYCFT              int64
HostRCFT               int64
GuestRCFT              int64
HostAttacksFT          int64
GuestAttacksFT         int64
HostDAttacksFT         int64
GuestDAttacksFT        int64
HostonTargetFT         int64
GuestonTargetFT        int64
HostoffTargetFT        int64
GuestoffTargetFT       int64
HostPossessionFT       int64
GuestPossessionFT      int64
dtype: object

# Visualizations

In [9]:
rf = RandomForestClassifier()
rf = rf.fit(X_train,y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



In [10]:
#Neuronal network
tf.keras.backend.clear_session() # For easy reset of notebook state.
#### Ignacio: You MUST explain a bit your neural network topology: 
#### One hidden layer with 128 neurons with the "Relux" activation function 
#### which is better for computational efficiency. Then you add an output layer with 3 neurons (one for each building you want to predict).
#### As the output neurons give you some value between 0-1, you apply the softmax function in order to normalize those outputs and interpret them 
#### as probabilities of being in each building.
nmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])

#### Ignacio: Why this choices of optimizer and loss function and metrics?
nmodel.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
nmodel.fit(np.array(X_train), np.array(y_train), epochs=10)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 623 samples
Epoch 1/10
623/623 [==============================] - 0s 667us/sample - loss: 0.8664 - acc: 0.6501
Epoch 2/10
623/623 [==============================] - 0s 51us/sample - loss: 0.6651 - acc: 0.6742
Epoch 3/10
623/623 [==============================] - 0s 53us/sample - loss: 0.6021 - acc: 0.6966
Epoch 4/10
623/623 [==============================] - 0s 54us/sample - loss: 0.5563 - acc: 0.7223
Epoch 5/10
623/623 [==============================] - 0s 58us/sample - loss: 0.5736 - acc: 0.7159
Epoch 6/10
623/623 [==============================] - 0s 56us/sample - loss: 0.5515 - acc: 0.7271
Epoch 7/10
623/623 [==============================] - 0s 67us/sample - loss: 0.5496 - acc: 0.7368
Epoch 8/10
623/623 [==============================] - 0s 65us/sample - loss: 0.5976 - acc: 0.7014
Epoch 9/10
623/623 [==============================] - 0s 53us/sample - loss: 0.5285 - acc: 0.7657
Epoch 10/10
623

In [11]:
test_loss, test_acc = nmodel.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

267/267 - 0s - loss: 0.7204 - acc: 0.7041

Test accuracy: 0.70411986


In [0]:
rfpreds = rf.predict(X_test)

In [0]:
npreds = nmodel.predict(X_test)

In [0]:
nnpreds = []
for i in range(0,len(npreds)):
    nnpreds.append(np.argmax(npreds[i]))

In [0]:
probs = rf.predict_proba(X_test)
rprobs = []
for i in probs:
    rprobs.append(i[1])
nprobs = []
for i in npreds:
    nprobs.append(i[1])

In [16]:
importances = rf.feature_importances_
cols = X.columns
pd.DataFrame({"Variable": cols,"Importance" : importances})

,Variable,Importance
0,ExpectedGoalsFT,0.053048
1,ExpectedGoalsHT,0.034931
2,HostCornersFT,0.041787
3,GuestCornersFT,0.055917
4,HostYCFT,0.020094
5,GuestYCFT,0.032862
6,HostRCFT,0.002218
7,GuestRCFT,0.001259
8,HostAttacksFT,0.120823
9,GuestAttacksFT,0.069286


In [17]:
accuracy_score(y_test, rfpreds)


0.6853932584269663

In [18]:
accuracy_score(y_test,nnpreds)

0.704119850187266

In [19]:
cohen_kappa_score(y_test, rfpreds)

0.17737676056338025

In [20]:
cohen_kappa_score(y_test,nnpreds)

-0.00745092420117488

In [21]:
roc_curve(y_test,rfpreds)

(array([0.        , 0.66666667, 1.        ]),
 array([0.        , 0.83068783, 1.        ]),
 array([2, 1, 0]))

In [22]:
roc_curve(y_test,nnpreds)

(array([0., 1., 1.]),
 array([0.        , 0.99470899, 1.        ]),
 array([2, 1, 0]))

In [23]:
confusion_matrix(y_test, rfpreds)

array([[ 26,  52],
       [ 32, 157]])

In [24]:
confusion_matrix(y_test,nnpreds)

array([[  0,  78],
       [  1, 188]])